# Hello, Data!
**My Python-MONAN-data first attempt.**

---
**Load libraries**

In [1]:
from matplotlib.tri import Triangulation
import xarray as xr

---
**Load data**

In [9]:
path = "/p/projetos/monan_atm/eduardo.eras/scripts_CD-CT/dataout/2025102500/Model/MONAN_DIAG_G_MOD_GFS_2025102500_2025102600.00.00.x40962L55.nc"

data = xr.open_dataset(path, engine = "netcdf4")
print(data.variables)

Frozen({'initial_time': <xarray.Variable ()>
[1 values with dtype=|S64]
Attributes:
    units:      YYYY-MM-DD_hh:mm:ss
    long_name:  Model initialization time, 'xtime': <xarray.Variable (Time: 1)>
[1 values with dtype=|S64]
Attributes:
    units:      YYYY-MM-DD_hh:mm:ss
    long_name:  Model valid time, 'Time': <xarray.IndexVariable 'Time' (Time: 1)>
array(['2025-10-26T00:00:00.000000000'], dtype='datetime64[ns]')
Attributes:
    long_name:      CF-compliant valid time
    standard_name:  time, 't_iso_levels': <xarray.Variable (nIsoLevelsT: 18)>
[18 values with dtype=float32]
Attributes:
    units:      Pa
    long_name:  Levels for vertical interpolation of temperature to isobaric ..., 'zgeo_isobaric': <xarray.Variable (Time: 1, nCells: 40962, t_iso_levels: 18)>
[737316 values with dtype=float32]
Attributes:
    units:      m
    long_name:  Geopotential height interpolated to isobaric surfaces defined..., 'temperature_isobaric': <xarray.Variable (Time: 1, nCells: 40962, t_iso_lev